In [ ]:
# library information:

import os
# It is the fundamental library of python, used to perform scientific computing. 
# It provides high-performance multidimensional arrays and tools to deal with them.
import numpy as np
# Pandas provide high performance, fast, easy to use data structures and data analysis tools for manipulating 
# numeric data and time series. Pandas is built on the numpy library and written in languages like Python.
import pandas as pd
# import library to read date range for stock ticker historical data range
import datetime
import json
from pathlib import Path
import alpaca_trade_api as tradeapi

from dotenv import load_dotenv
# import ipynb.fs.full
# https://stackoverflow.com/questions/44116194/import-a-function-from-another-ipynb-file
# from ipynb.fs.full.test_sr import *
# from ipynb.fs.full.date_functions import *
# from ipynb.fs.full.my_functions import *

# since functions will be in a different folder, import library to read path info
import sys
sys.path.append('Saeed/functions/')
sys.path.append('Saeed/functions/utils/')
# and now import my functions
from test_sr import *
from date_functions import *
from my_functions import *

# ===================================================================================================
# http://theautomatic.net/yahoo_fin-documentation/
# pip install yahoo-fin
# import yahoo_fin.stock_info as si

# import data libraries to read date range for stock ticker historical data range
# from datetime import date

# other libraries
# import pandas_datareader as pdr
%matplotlib inline
import yfinance as yf

In [ ]:
# INITIAL: Get Data

# SAM: Get S&P 500 tickers
sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500_tickers = sp500_tickers.Symbol.to_list()
# print(sp500_tickers)

In [ ]:
# INITIAL: Get Data

# SAM: Clean Tickers of periods
sp500_tickers = [x.replace(".", "-") for x in sp500_tickers]
# print(sp500_tickers)

In [ ]:
# INITIAL: Get Data

# decipher time
# format current date as ISO format
this_moment = datetime.datetime.now()
this_moment

In [ ]:
# INITIAL: Get Data

# decipher time
# number of days for which we need data. for 200SMA we should atleast get 250 days
num_of_days = 30    # 365 - Get atleast a year's worth of data.
num_of_days

In [ ]:
# INITIAL: Get Data

# decipher time
# start date
start_date = get_begin_date(this_moment, num_of_days)
start_date

In [ ]:
# INITIAL: Get Data

# decipher time
# get the end date for the date range
finish_date = str(this_moment).split()[0]
#finish_date
begin_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
#begin_date
end_date = pd.Timestamp(finish_date, tz="America/New_York").isoformat()
#end_date

In [ ]:
# INITIAL: Get Data

# get tickers for "dow" or "sp500" or "nasdaq" one at a time

# dow30 has 30 symbols
tickers_dow = get_tickers("dow")
choice = "dow"
# tickers_dow

# sp500 has 505 symbols
# tickers_sp500 = get_tickers("sp500")
# choice = "sp500"
# tickers_sp500

# nasdaq has 4942 symbols
# tickers_nasdaq = get_tickers("nasdaq")
# choice = "nasdaq"
# tickers_nasdaq

# if we decide to use more than one market index, we must remove duplicates
# all_tickers = list(set(tickers_dow + tickers_sp500)) 

# all_tickers = tickers_dow
all_tickers = sp500_tickers
# all_tickers = tickers_sp500
# all_tickers = tickers_nasdaq
# all_tickers = ["AAPL", "IBM", "WMT"]     # ["AAPL", "V", "WMT"]
# choice = "sample"

# print(all_tickers)
# type(all_tickers)
all_tickers

In [ ]:
selection = yf.download((all_tickers), period="1y", group_by="ticker")
# 70.47 seconds
selection

In [ ]:
# INITIAL: Get Data: Save File

# save the data
# json = final_df("hist_data.json")
# save the file for future read
with open('resources/data/hist_data.json', 'w') as outfile:
     json.dump(selection.to_json(), outfile)

In [ ]:
# PROCESS Data

current_price = []
dept_to_equity = []
total_debt = []
total_revenue = []
market_cap = []

print("..... Building Indicator, Please Wait .....")
print()

count = 1
for ticker in all_tickers:
    print("count: " + str(count) + "  -  ticker: " + ticker)
    info = yf.Ticker(ticker).info

    current_price.append(info["currentPrice"])
    dept_to_equity.append(info["debtToEquity"])
    total_debt.append(info["totalDebt"])
    total_revenue.append(info["totalRevenue"])
    market_cap.append(info["marketCap"])

    count+=1

# print(current_price)
# print(dept_to_equity)
# print(total_debt)
# print(total_revenue)
# print(market_cap)

df_cp = pd.DataFrame(data=current_price, index=all_tickers, columns=["Current Price"])
df_dte = pd.DataFrame(data=dept_to_equity, index=all_tickers, columns=["Debt To Equity"])
df_td = pd.DataFrame(data=total_debt, index=all_tickers, columns=["Total Debt"])
df_tr = pd.DataFrame(data=total_revenue, index=all_tickers, columns=["Total Revenue"])
df_cmc = pd.DataFrame(data=market_cap, index=all_tickers, columns=["Market Cap"])

# print(df_cp)
# print(df_dte)
# print(df_td)
# print(df_tr)
# print(df_cmc)

In [ ]:
# yf.Ticker("BAX").info